In [0]:
#  https://www.kaggle.com/lpanda/netflix-data

In [1]:
from pyspark.sql import *
%load_ext autoreload
%autoreload 1
import matplotlib.pyplot as plt
%aimport experiment
%aimport privacy_spark

In [2]:
spark = SparkSession \
    .builder \
    .appName("Privacy Project") \
    .getOrCreate()

In [91]:
exp = experiment.Experiment(spark)
exp.load_dataset("./ratings.csv")
# exp.load_dataset("./ratings.csv", nrows=100000)

In [4]:
no_info = privacy_spark.Auxiliary(False, False, 0, 0)
ratings_exact_date_3  = privacy_spark.Auxiliary(True, True, 0, 3)
ratings_exact_date_14 = privacy_spark.Auxiliary(True, True, 0, 14)

aux_list = 3*[ratings_exact_date_14] + 1*[no_info]
print(exp.evaluate(aux_list, N=100))

Py4JJavaError: An error occurred while calling o69.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 6.0 failed 1 times, most recent failure: Lost task 5.0 in stage 6.0 (TID 68, localhost, executor driver): java.io.IOException: Aucun espace disponible sur le périphérique
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:332)
	at java.nio.channels.Channels.writeFullyImpl(Channels.java:78)
	at java.nio.channels.Channels.writeFully(Channels.java:101)
	at java.nio.channels.Channels.access$000(Channels.java:61)
	at java.nio.channels.Channels$1.write(Channels.java:174)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	a

In [ ]:
scoring.matching_set(scores, 0.5)
# scoring.output(scores, 0.5)

### Experiment without Movie

In [64]:
no_info = privacy_spark.Auxiliary(False, False, 0, 0)
ratings_exact_date_3  = privacy_spark.Auxiliary(True, True, 0, 3)
ratings_exact_date_14 = privacy_spark.Auxiliary(True, True, 0, 14)
ratings_exact_date_0  = privacy_spark.Auxiliary(True, True, 0, 0)

aux_list = [ratings_exact_date_0]

In [65]:
aux = exp.generate_auxiliary_data(aux_list, 1)
exp.compute_score(aux, similarity='netflix', with_movie=False).sort('value', ascending=False).show()

# exp.evaluate(aux_list, N=1, similarity='netflix', with_movie=False)

+--------+------------------+
|custId_2|             value|
+--------+------------------+
|  885013|               2.0|
|    4326|1.7165313105737892|
|  808731|1.6483443410015097|
| 1308744|1.2299484296063814|
| 1817215|1.2018965179946555|
|   93986|1.1617614600341017|
|  883478|1.1399224880243093|
|   30878|1.1002588437228038|
|  729846|1.0937928486138762|
| 2647871|1.0877438650242943|
| 1665054|1.0301973834223186|
|  814701| 1.026834238065184|
|  401047|1.0100518357446335|
| 1838912|1.0100024228240017|
| 1972040|1.0097222973716102|
| 1011918|  1.00822974704902|
| 1503895|1.0060967465655157|
| 1711859|1.0042252930202749|
| 1159695|1.0002839629839033|
|  573537|1.0001903460804424|
+--------+------------------+
only showing top 20 rows



In [66]:
exp.df.schema

StructType(List(StructField(movieId,IntegerType,true),StructField(custId,IntegerType,true),StructField(rating,IntegerType,true),StructField(days,LongType,true)))